<a href="https://colab.research.google.com/github/nekoniii3/GPT_function_calling/blob/main/function_calling_%E3%82%A2%E3%83%8A%E3%83%AA%E3%82%B9%E3%83%881%E3%81%8B%E6%9C%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OpenAIパッケージのインストール
!pip install openai

In [ ]:
import openai

openai.api_key = "ここをOpenAIキーに入れ替えてください。" #@param {type:"string"}


In [ ]:
import pandas_datareader.data as web

# DataReaderでstooqから株価情報を取得する関数（開始日～終了日）
def get_stock_price(code, date_from ,date_to):

  # 日本用のコードに変換
  code = code + '.JP'

  # stooqからデータ取得
  df = web.DataReader(code, data_source='stooq', start=date_from,end=date_to)

  # 日付と株価の終値を返す ※日付形式がおかしくなるのでorient='table'を利用
  return df['Close'].to_json(orient='table')

# Chat-GPTに伝えるための関数の仕様を記したのJSON
stock_function =  {
    "name": "get_stock_price",
      "description": "証券コードと2つの日付を渡すと指定の会社のその2つの日付の間の株価の終値を返します",
      "parameters": {
          "type": "object",
          "properties": {
              "code": {
                  "type": "string",
                  "description": "証券コード",
              },
              "date_from": {
                  "type": "string",
                  "description": "株価を知りたい日付の開始日",
              },
              "date_to": {
                  "type": "string",
                  "description": "株価を知りたい日付の終了日",
              },
          },
          "required": ["code", "date_from", "date_to"],
      },
}


In [ ]:
import json

# GPTへ1回目の問い合わせ
prompt = 'カルビー株式会社の2023年6月の株価の情報と、その動きについて分析してください。'

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "system", "content": "You are a capable securities analyst."}
     ,{"role": "user", "content": prompt}],
    functions=[stock_function],
    function_call="auto",
)

message = response["choices"][0]["message"]

# 回答にfunction_callの指示があった場合
if message.get("function_call"):

    # GPTの回答からget_stock_priceを実行
    function_name = message["function_call"]["name"]
    arguments=json.loads(message["function_call"]["arguments"])
    function_response = get_stock_price(
        code=arguments.get("code"),
        date_from=arguments.get("date_from"),
        date_to=arguments.get("date_to")
    )

    # GPTへ2回目の問い合わせ
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": prompt},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    print(second_response.choices[0]["message"]["content"])